In [13]:
# this is a program that estimates a maximum likelihood model 
# we are estimating three parameters
# k is the elo k parameter
# hfa is the home field advantage parameter. This gets added to 
# mr is a mean reversion parameter.  

# this reads in the nfl historical data since 2000.  
# this reads in odds data
# then it tells you how to bet based on the elo model and the odds. 



import pandas as pd
import numpy as np
import os



# Directory where the play-by-play CSV files are stored
directory_path = '../ELO/PBP'

# Create an empty list to store DataFrames
dfs = []

# Loop through the years 2000 to 2024
for year in range(2000, 2025):
    file_path = os.path.join(directory_path, f'nfl_{year}.csv')
    if os.path.exists(file_path):  # Check if the file exists
        try:
            # Read the CSV file
            df = pd.read_csv(file_path, low_memory=False)
            df['year'] = year
            dfs.append(df)  # Append the DataFrame to the list
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    else:
        print(f"File not found: {file_path}")

# Concatenate all DataFrames into one DataFrame
if dfs:
    combined_df = pd.concat(dfs, ignore_index=True)
    print("Successfully combined DataFrames from 2000 to 2024.")
else:
    print("No files were processed.")

print(combined_df.columns.to_list)
cdf = combined_df[combined_df.desc == 'END GAME'][['home_team','away_team','year','week','total_home_score','total_away_score']]
cdf['winner'] = np.where(cdf.total_home_score == cdf.total_away_score,'TIE',np.nan)
cdf['winner'] = np.where(cdf.total_home_score > cdf.total_away_score,cdf['home_team'],cdf['winner'])
cdf['winner'] = np.where(cdf.total_home_score < cdf.total_away_score,cdf['away_team'],cdf['winner'])
cdf = cdf[['home_team','away_team','year','week','winner']]
cdf.columns = ['home', 'vis', 'year','week','winner']
cdf = cdf.sort_values(by=['year','week'], ascending=[True,True])
elo_dict = {key: 1500 for key in cdf['home'].unique().tolist()}

#cdf.to_csv('./nfl_outcomes.csv', index=False)

Successfully combined DataFrames from 2000 to 2024.
<bound method IndexOpsMixin.tolist of Index(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       ...
       'offense_players', 'defense_players', 'n_offense', 'n_defense',
       'ngs_air_yards', 'time_to_throw', 'was_pressure', 'route',
       'defense_man_zone_type', 'defense_coverage_type'],
      dtype='object', length=391)>


In [14]:
import pandas as pd
import numpy as np

# --- Extract game-level data ---
game_df = combined_df[combined_df.desc == 'END GAME'][[
    'home_team', 'away_team', 'year', 'week', 'game_id', 'game_date',
    'total_home_score', 'total_away_score'
]]

# --- Identify QB Plays (Pass + Sacks + QB Runs) ---
qb_plays = combined_df[
    ((combined_df.qb_dropback == 1) |  # Includes Passes + Sacks
     ((combined_df.rush_attempt == 1) & (combined_df.rusher_player_id.notna()) &  # Includes QB Rushes
      (combined_df.rusher_player_id == combined_df.passer_player_id)))
].copy()

# --- Ensure rushing plays are included correctly ---
qb_plays['qb_player_id'] = np.where(qb_plays.passer_player_id.notna(), 
                                    qb_plays.passer_player_id, qb_plays.rusher_player_id)

qb_plays['qb_player_name'] = np.where(qb_plays.passer_player_name.notna(), 
                                      qb_plays.passer_player_name, qb_plays.rusher_player_name)

# --- Count Total QB Snaps per Game (Passes + Sacks + QB Runs) ---
qb_snaps = qb_plays.groupby(['game_id', 'posteam', 'qb_player_id']) \
    .agg(qb_snaps=('play_id', 'count')).reset_index()

# --- Keep Only QBs Who Played at Least 10 Snaps ---
qb_snaps = qb_snaps[qb_snaps.qb_snaps >= 10]

# --- Determine Starting QB (Most Snaps per Game) ---
starting_qbs = qb_snaps.sort_values(by=['game_id', 'posteam', 'qb_snaps'], ascending=[True, True, False]) \
    .groupby(['game_id', 'posteam']).first()[['qb_player_id', 'qb_snaps']].reset_index()

# --- Merge QB Names ---
starting_qbs = starting_qbs.merge(qb_plays[['game_id', 'posteam', 'qb_player_id', 'qb_player_name']].drop_duplicates(),
                                   on=['game_id', 'posteam', 'qb_player_id'], how='left')

# --- Compute **Total QB EPA/play (Pass + Rush + Sack)** ---
qb_epa = qb_plays.groupby(['game_id', 'posteam', 'qb_player_id']) \
    .agg(
        total_qb_epa=('epa', 'sum'),   # Total EPA (Pass + Sack + Run)
        total_qb_plays=('play_id', 'count')  # Total plays (passes + sacks + runs)
    ).reset_index()

qb_epa['qb_epa_per_play'] = qb_epa['total_qb_epa'] / qb_epa['total_qb_plays']

# --- Merge QB EPA with Starting QBs ---
starting_qbs = starting_qbs.merge(qb_epa, on=['game_id', 'posteam', 'qb_player_id'], how='left')

# --- Merge QB Data with Game Data ---
game_df = game_df.merge(starting_qbs, left_on=['game_id', 'home_team'], right_on=['game_id', 'posteam']).rename(
    columns={'qb_player_name': 'home_qb', 'qb_player_id': 'home_qb_id', 'qb_snaps': 'home_qb_snaps', 'qb_epa_per_play': 'home_qb_epa'}
).drop(columns=['posteam'])

game_df = game_df.merge(starting_qbs, left_on=['game_id', 'away_team'], right_on=['game_id', 'posteam']).rename(
    columns={'qb_player_name': 'away_qb', 'qb_player_id': 'away_qb_id', 'qb_snaps': 'away_qb_snaps', 'qb_epa_per_play': 'away_qb_epa'}
).drop(columns=['posteam'])

# --- Compute Defensive EPA as Negative of Opposing QB EPA ---
game_df['home_def_epa'] = -game_df['away_qb_epa']
game_df['away_def_epa'] = -game_df['home_qb_epa']

# --- Keep only the relevant columns ---
game_df = game_df[['game_id', 'game_date', 'year', 'week', 'home_team', 'away_team', 'home_qb', 'away_qb',
                   'home_qb_epa', 'away_qb_epa', 'home_qb_snaps', 'away_qb_snaps',
                   'home_def_epa', 'away_def_epa', 'total_home_score', 'total_away_score']]

# Display first few rows
game_df.head()


,game_id,game_date,year,week,home_team,away_team,home_qb,away_qb,home_qb_epa,away_qb_epa,home_qb_snaps,away_qb_snaps,home_def_epa,away_def_epa,total_home_score,total_away_score
0,2000_01_ARI_NYG,2000-09-03,2000,1,NYG,ARI,K.Collins,J.Plummer,-0.009584,0.042953,24,54,-0.042953,0.009584,21.0,16.0
1,2000_01_BAL_PIT,2000-09-03,2000,1,PIT,BAL,K.Graham,T.Banks,-0.198571,0.087666,39,33,-0.087666,0.198571,0.0,16.0
2,2000_01_CAR_WAS,2000-09-03,2000,1,WAS,CAR,B.Johnson,S.Beuerlein,0.147979,-0.145791,35,33,0.145791,-0.147979,20.0,17.0
3,2000_01_CHI_MIN,2000-09-03,2000,1,MIN,CHI,D.Culpepper,C.McNown,0.343594,0.205944,28,49,-0.205944,-0.343594,30.0,27.0
4,2000_01_DEN_STL,2000-09-04,2000,1,LA,DEN,K.Warner,B.Griese,0.383447,0.320759,39,33,-0.320759,-0.383447,41.0,36.0


In [15]:
import pandas as pd
import numpy as np

# --- Extract game-level data ---
game_df = combined_df[combined_df.desc == 'END GAME'][[
    'home_team', 'away_team', 'year', 'week', 'game_id', 'game_date',
    'total_home_score', 'total_away_score'
]]

# --- Identify QB Plays (Pass + Sacks + QB Runs) ---
qb_plays = combined_df[
    ((combined_df.qb_dropback == 1) |  # Includes Passes + Sacks
     ((combined_df.rush_attempt == 1) & (combined_df.rusher_player_id.notna()) &  # Includes QB Rushes
      (combined_df.rusher_player_id == combined_df.passer_player_id)))
].copy()

# --- Ensure rushing plays are included correctly ---
qb_plays['qb_player_id'] = np.where(qb_plays.passer_player_id.notna(), 
                                    qb_plays.passer_player_id, qb_plays.rusher_player_id)

qb_plays['qb_player_name'] = np.where(qb_plays.passer_player_name.notna(), 
                                      qb_plays.passer_player_name, qb_plays.rusher_player_name)

# --- Count Total QB Snaps per Game (Passes + Sacks + QB Runs) ---
qb_snaps = qb_plays.groupby(['game_id', 'posteam', 'qb_player_id']) \
    .agg(qb_snaps=('play_id', 'count')).reset_index()

# --- Keep Only QBs Who Played at Least 10 Snaps ---
qb_snaps = qb_snaps[qb_snaps.qb_snaps >= 10]

# --- Determine Starting QB (Most Snaps per Game) ---
starting_qbs = qb_snaps.sort_values(by=['game_id', 'posteam', 'qb_snaps'], ascending=[True, True, False]) \
    .groupby(['game_id', 'posteam']).first()[['qb_player_id', 'qb_snaps']].reset_index()

# --- Merge QB Names ---
starting_qbs = starting_qbs.merge(qb_plays[['game_id', 'posteam', 'qb_player_id', 'qb_player_name']].drop_duplicates(),
                                   on=['game_id', 'posteam', 'qb_player_id'], how='left')

# --- Compute **Total QB EPA/play (Pass + Rush + Sack)** ---
qb_epa = qb_plays.groupby(['game_id', 'posteam', 'qb_player_id']) \
    .agg(
        total_qb_epa=('epa', 'sum'),   # Total EPA (Pass + Sack + Run)
        total_qb_plays=('play_id', 'count')  # Total plays (passes + sacks + runs)
    ).reset_index()

qb_epa['qb_epa_per_play'] = qb_epa['total_qb_epa'] / qb_epa['total_qb_plays']

# --- Merge QB EPA with Starting QBs ---
starting_qbs = starting_qbs.merge(qb_epa, on=['game_id', 'posteam', 'qb_player_id'], how='left')

# --- Compute Defensive EPA/play ---
def_epa = combined_df.groupby(['game_id', 'defteam']).agg(
    total_def_epa=('epa', 'sum'),
    total_def_plays=('play_id', 'count')
).reset_index()
def_epa['def_epa_per_play'] = def_epa['total_def_epa'] / def_epa['total_def_plays']

# --- Merge QB & Defensive Data ---
game_df = game_df.merge(starting_qbs, left_on=['game_id', 'home_team'], right_on=['game_id', 'posteam']).rename(
    columns={'qb_player_name': 'home_qb', 'qb_player_id': 'home_qb_id', 'qb_snaps': 'home_qb_snaps', 'qb_epa_per_play': 'home_qb_epa'}
).drop(columns=['posteam'])

game_df = game_df.merge(starting_qbs, left_on=['game_id', 'away_team'], right_on=['game_id', 'posteam']).rename(
    columns={'qb_player_name': 'away_qb', 'qb_player_id': 'away_qb_id', 'qb_snaps': 'away_qb_snaps', 'qb_epa_per_play': 'away_qb_epa'}
).drop(columns=['posteam'])

# Merge Defensive EPA/play
game_df = game_df.merge(def_epa, left_on=['game_id', 'home_team'], right_on=['game_id', 'defteam']).rename(
    columns={'def_epa_per_play': 'home_def_epa'}
).drop(columns=['defteam'])

game_df = game_df.merge(def_epa, left_on=['game_id', 'away_team'], right_on=['game_id', 'defteam']).rename(
    columns={'def_epa_per_play': 'away_def_epa'}
).drop(columns=['defteam'])

# --- Keep only the relevant columns ---
game_df = game_df[['game_id', 'game_date', 'year', 'week', 'home_team', 'away_team', 'home_qb', 'away_qb',
                   'home_qb_epa', 'away_qb_epa', 'home_qb_snaps', 'away_qb_snaps',
                   'home_def_epa', 'away_def_epa', 'total_home_score', 'total_away_score']]

game_df.head()


,game_id,game_date,year,week,home_team,away_team,home_qb,away_qb,home_qb_epa,away_qb_epa,home_qb_snaps,away_qb_snaps,home_def_epa,away_def_epa,total_home_score,total_away_score
0,2000_01_ARI_NYG,2000-09-03,2000,1,NYG,ARI,K.Collins,J.Plummer,-0.009584,0.042953,24,54,0.019156,0.062846,21.0,16.0
1,2000_01_BAL_PIT,2000-09-03,2000,1,PIT,BAL,K.Graham,T.Banks,-0.198571,0.087666,39,33,0.030901,-0.203437,0.0,16.0
2,2000_01_CAR_WAS,2000-09-03,2000,1,WAS,CAR,B.Johnson,S.Beuerlein,0.147979,-0.145791,35,33,0.073094,0.072648,20.0,17.0
3,2000_01_CHI_MIN,2000-09-03,2000,1,MIN,CHI,D.Culpepper,C.McNown,0.343594,0.205944,28,49,0.152655,0.204863,30.0,27.0
4,2000_01_DEN_STL,2000-09-04,2000,1,LA,DEN,K.Warner,B.Griese,0.383447,0.320759,39,33,0.159031,0.180758,41.0,36.0


In [16]:
game_df.tail(10)

,game_id,game_date,year,week,home_team,away_team,home_qb,away_qb,home_qb_epa,away_qb_epa,home_qb_snaps,away_qb_snaps,home_def_epa,away_def_epa,total_home_score,total_away_score
6694,2024_19_MIN_LA,2025-01-13,2024,19,LA,MIN,M.Stafford,S.Darnold,0.183461,-0.532014,29,51,-0.231407,0.011908,27.0,9.0
6695,2024_19_PIT_BAL,2025-01-11,2024,19,BAL,PIT,L.Jackson,R.Wilson,0.431245,0.195057,25,35,0.076494,0.199139,28.0,14.0
6696,2024_19_WAS_TB,2025-01-12,2024,19,TB,WAS,B.Mayfield,J.Daniels,0.526682,0.301694,23,40,0.150851,0.119288,20.0,23.0
6697,2024_20_BAL_BUF,2025-01-19,2024,20,BUF,BAL,J.Allen,L.Jackson,-0.075263,0.103991,23,33,0.129496,0.109037,27.0,25.0
6698,2024_20_HOU_KC,2025-01-18,2024,20,KC,HOU,P.Mahomes,C.Stroud,0.018582,0.106800,31,41,-0.038002,0.099001,23.0,14.0
6699,2024_20_LA_PHI,2025-01-19,2024,20,PHI,LA,J.Hurts,M.Stafford,-0.134019,0.072110,30,50,-0.012526,0.070775,28.0,22.0
6700,2024_20_WAS_DET,2025-01-18,2024,20,DET,WAS,J.Goff,J.Daniels,-0.040606,0.560546,40,37,0.251547,0.060502,31.0,45.0
6701,2024_21_BUF_KC,2025-01-26,2024,21,KC,BUF,P.Mahomes,J.Allen,0.661697,0.224212,35,38,0.153403,0.188303,32.0,29.0
6702,2024_21_WAS_PHI,2025-01-26,2024,21,PHI,WAS,J.Hurts,J.Daniels,0.501557,-0.120930,31,57,-0.075557,0.278418,55.0,23.0
6703,2024_22_KC_PHI,2025-02-09,2024,22,PHI,KC,J.Hurts,P.Mahomes,0.658059,-0.366555,29,44,-0.262756,0.079915,40.0,22.0


In [17]:
# Check for missing QB values
missing_qb_games = game_df[(game_df["home_qb"].isna()) | (game_df["away_qb"].isna())]

# Display the results
if missing_qb_games.empty:
    print("✅ No missing QB data. Every game has recorded starting QBs.")
else:
    print("⚠️ Missing QB data detected! Here are the games with missing values:")
    missing_qb_games


✅ No missing QB data. Every game has recorded starting QBs.


In [5]:
game_df.to_csv('./nfl_outcomes.csv', index=False)